In [1]:
#on travaille sur le json issu du scraping de jp
import pandas as pd
import json

#chemin du json
json_file_path = 'melo/jpboxoffice/moviesdata.json'

# charger json
with open(json_file_path) as f:
    data = json.load(f)

# json devient df
films = pd.json_normalize(data)

#  df convertit en CSV
films.to_csv('moviedata_test.csv', index=False)
films.dtypes

url            object
titre          object
realisateur    object
pays           object
date           object
genre          object
studio         object
casting        object
franchise      object
remake         object
budget         object
acteurs        object
producteur     object
compositeur    object
dtype: object

In [2]:
#normalisation à la main (même principe que minmaxscaler)
actors = pd.read_csv('acteurs.csv')

actors['prix_normal'] = (actors['prix'] - actors['prix'].min()) / (actors['prix'].max() - actors['prix'].min())

print(actors)
actors['prix_normal'].max()

     duree_carriere                  name  nombre_tournage  prix  prix_normal
0                40     Leonardo DiCaprio               72     8     0.241379
1                39           Johnny Depp               97     9     0.275862
2                43            Tom Cruise               66    18     0.586207
3                26         Jason Statham               59     2     0.034483
4                16         Margot Robbie               47    15     0.482759
..              ...                   ...              ...   ...          ...
268              50     Denzel Washington               70     8     0.241379
269              28         Jeremy Renner               55     7     0.206897
270              30       Jean-Paul Rouve               79     4     0.103448
271              21  Matthias Schoenaerts               42     5     0.137931
272              59     Samuel L. Jackson              168     4     0.103448

[273 rows x 5 columns]


1.0

In [3]:
import pandas as pd
df = pd.read_csv('moviesdatafinal.csv')

df.head(2)

,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre,url
0,NaN,12 547 392 €,NaN,27/02/2008,\n 1h 46min ...,4378720,NaN,Comédie,France,NaN,\n Dany Boon,Remake,793,\n Pathé,\n Bienvenue chez l...,https://www.jpbox-office.com/fichfilm.php?id=1...
1,"\n Dany Boon,\n ...",12 547 392 €,NaN,27/02/2008,\n 1h 46min ...,4378720,NaN,Comédie,France,Claude Berri,\n Dany Boon,Remake,793,\n Pathé,\n Bienvenue chez l...,https://www.jpbox-office.com/fichfilm.php?id=1...


In [4]:
# Dropping 'url' and 'salles_premiere_semaine' columns
movies_data_cleaned = df.drop(['url'], axis=1)

# Keeping rows where both 'acteurs' and 'budget' are not null to avoid incomplete data
movies_data_cleaned = movies_data_cleaned.dropna(subset=['acteurs', 'budget'])

#Dropping duplicate rows based on all columns to ensure unique movie entries
movies_data_cleaned = movies_data_cleaned.drop_duplicates()

movies_data_cleaned.head(2)

movies_data_cleaned.shape

(4549, 15)

In [5]:
###NETTOYAGE REALISATEUR

for index, value in movies_data_cleaned['realisateur'].items():
    if isinstance(value,str):
        value = value.replace('\r\n','')
        value = value.replace('-','')
        value = value.strip()
        movies_data_cleaned.at[index, 'realisateur'] = value

movies_data_cleaned.head(5)

,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre
1,"\n Dany Boon,\n ...",12 547 392 €,NaN,27/02/2008,\n 1h 46min ...,4378720,NaN,Comédie,France,Claude Berri,Dany Boon,Remake,793,\n Pathé,\n Bienvenue chez l...
5,\n Christian Clavie...,50 300 000 €,NaN,30/01/2002,\n 1h 47min ...,3685097,Franchise,Comédie,France,Claude Berri,Alain Chabat,NaN,945,\n Pathé,\n Astérix et Obéli...
6,"\n Ewan McGregor,\n...",113 000 000 $,John Williams,18/05/2005,\n 2h 26min ...,3303005,Franchise,Fantasy,Etats-Unis,George Lucas,George Lucas,NaN,938,\n 20th Century Fox,\n La Revanche des ...
7,"\n Chris Evans,\n ...",356 000 000 $,Alan Silvestri,24/04/2019,\n 3h 01min ...,3426471,Franchise,Comicbook,Etats-Unis,Kevin Feige,Russo (brothers),NaN,633,\n Walt Disney Pictures,\n Avengers: Endgame
10,"\n Sam Worthington,...",350 000 000 $,NaN,14/12/2022,\n 3h 12min ...,2739848,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,762,\n Walt Disney Pictures,\n Avatar : la voie...


In [6]:
###NETTOYAGE TITRE
for index, value in movies_data_cleaned['titre'].items():
    if isinstance(value,str):
        value = value.replace('\r\n','')
        value = value.strip()
        movies_data_cleaned.at[index, 'titre'] = value

###NETTOYAGE STUDIO
for index, value in movies_data_cleaned['studio'].items():
    if isinstance(value,str):
        value = value.replace('\r\n','')
        value = value.strip()
        movies_data_cleaned.at[index, 'studio'] = value

###NETTOYAGE ACTEURS
movies_data_cleaned['acteurs'] = movies_data_cleaned['acteurs'].str.strip().replace(r"\r\n", "", regex=True).replace(r"\s+", " ", regex=True)

In [7]:
import pandas as pd

# Supprimer les caractères \r\n et les espaces supplémentaires
def clean_duration(duration):
    return duration.replace('\r\n', '').strip()

# Convertir la durée en heures et minutes en minutes
def convert_to_minutes(duration):
    duration = duration.strip()  # Supprimer les espaces supplémentaires
    parts = duration.split()
    total_minutes = 0
    for part in parts:
        if 'h' in part:
            total_minutes += int(part[:-1]) * 60
        elif 'min' in part:
            total_minutes += int(part[:-3])
    return total_minutes


# Nettoyer la colonne de la durée
movies_data_cleaned['duree'] = movies_data_cleaned['duree'].apply(clean_duration)

# Convertir la durée en minutes
movies_data_cleaned['duree'] = movies_data_cleaned['duree'].apply(convert_to_minutes)

# Afficher le résultat
print(movies_data_cleaned.head())

                                              acteurs         budget  \
1   Dany Boon, Kad Merad, Michel Galabru, Jérôme C...   12 547 392 €   
5   Christian Clavier, Jamel Debbouze, Gérard Depa...   50 300 000 €   
6   Ewan McGregor, Natalie Portman, Samuel L. Jackson  113 000 000 $   
7   Chris Evans, Chris Hemsworth, Robert Downey Jr...  356 000 000 $   
10    Sam Worthington, Sigourney Weaver, Kate Winslet  350 000 000 $   

       compositeur          date  duree  entrees_premiere_semaine  franchise  \
1              NaN    27/02/2008    106                   4378720        NaN   
5              NaN    30/01/2002    107                   3685097  Franchise   
6    John Williams    18/05/2005    146                   3303005  Franchise   
7   Alan Silvestri    24/04/2019    181                   3426471  Franchise   
10             NaN    14/12/2022    192                   2739848  Franchise   

              genre        pays    producteur       realisateur  remake  \
1          

In [8]:
### NETTOYAGE DATE
movies_data_cleaned['date'] = movies_data_cleaned['date'].str.strip()
movies_data_cleaned['date'] = pd.to_datetime(movies_data_cleaned['date'], format='%d/%m/%Y')

movies_data_cleaned.head(2)

,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre
1,"Dany Boon, Kad Merad, Michel Galabru, Jérôme C...",12 547 392 €,NaN,2008-02-27,106,4378720,NaN,Comédie,France,Claude Berri,Dany Boon,Remake,793,Pathé,Bienvenue chez les Ch'tis
5,"Christian Clavier, Jamel Debbouze, Gérard Depa...",50 300 000 €,NaN,2002-01-30,107,3685097,Franchise,Comédie,France,Claude Berri,Alain Chabat,NaN,945,Pathé,Astérix et Obélix: Mission Cléopatre


In [9]:
### NETTOYAGE BUDGET
movies_data_cleaned['budget'] = movies_data_cleaned['budget'].str.replace('[\$,€]', '', regex=True).str.replace(' ', '').str.replace('?', '')

# Converting to numeric, coercing errors to NaN, then filling NaNs with 0
movies_data_cleaned['budget'] = pd.to_numeric(movies_data_cleaned['budget'], errors='coerce').fillna(0).astype(int)

movies_data_cleaned.head(2)

,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre
1,"Dany Boon, Kad Merad, Michel Galabru, Jérôme C...",12547392,NaN,2008-02-27,106,4378720,NaN,Comédie,France,Claude Berri,Dany Boon,Remake,793,Pathé,Bienvenue chez les Ch'tis
5,"Christian Clavier, Jamel Debbouze, Gérard Depa...",50300000,NaN,2002-01-30,107,3685097,Franchise,Comédie,France,Claude Berri,Alain Chabat,NaN,945,Pathé,Astérix et Obélix: Mission Cléopatre


In [10]:
movies_data_cleaned[movies_data_cleaned.acteurs.str.contains('Sam Worthington')]

,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre
10,"Sam Worthington, Sigourney Weaver, Kate Winslet",350000000,NaN,2022-12-14,192,2739848,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,762,Walt Disney Pictures,Avatar : la voie de l'eau
567,"Josh Brolin, Jake Gyllenhaal, Keira Knightley,...",55000000,NaN,2015-09-23,122,313160,NaN,Thriller,Etats-Unis,NaN,NaN,NaN,376,Universal,Everest (2015)
947,"Sam Worthington, Ed Harris",42000000,NaN,2012-02-15,102,156939,NaN,Thriller,Etats-Unis,Lorenzo di Bonaventura,NaN,NaN,254,Studiocanal,Dos au mur
1321,"Arnold Schwarzenegger, Sam Worthington",35000000,NaN,2014-05-07,109,88534,NaN,Aventure - Action,Etats-Unis,NaN,NaN,NaN,172,Metropolitan,Sabotage
1609,"Sam Worthington, Sigourney Weaver",25000000,James Horner,2010-09-01,160,53756,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,78,20th Century Fox,Avatar (Special Edition)
1928,"Jennifer Aniston, Sam Worthington, William H. ...",0,NaN,2015-04-08,102,24050,NaN,Drame,Etats-Unis,NaN,NaN,NaN,82,Warner Bros.,Cake
2697,Sam Worthington,20000000,NaN,2008-08-13,95,8527,NaN,Horreur,Australie,NaN,NaN,NaN,NaN,TFM Distribution,Solitaire
4688,"Bruce Willis, Colin Farrell, Sam Worthington",70000000,NaN,2002-05-29,125,84139,NaN,Guerre,Etats-Unis,NaN,NaN,NaN,151,UFD,Mission évasion
4777,"Jessica Chastain, Helen Mirren, Sam Worthington",20000000,Thomas Newman,2011-06-15,110,88044,NaN,Thriller,Etats-Unis,NaN,NaN,NaN,206,UIP,L'Affaire Rachel Singer
6609,"Keira Knightley, Guillaume Canet, Eva Mendes, ...",10000000,NaN,2011-02-16,93,187573,NaN,Drame,Etats-Unis,NaN,NaN,NaN,224,Gaumont,Last Night


In [11]:
###repérer les lignes avec sam worthington 
acteur_sam = movies_data_cleaned.mask(movies_data_cleaned["acteurs"] == "Sam Worthington")
acteur_sam

,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre
1,"Dany Boon, Kad Merad, Michel Galabru, Jérôme C...",12547392.0,NaN,2008-02-27,106.0,4378720.0,NaN,Comédie,France,Claude Berri,Dany Boon,Remake,793,Pathé,Bienvenue chez les Ch'tis
5,"Christian Clavier, Jamel Debbouze, Gérard Depa...",50300000.0,NaN,2002-01-30,107.0,3685097.0,Franchise,Comédie,France,Claude Berri,Alain Chabat,NaN,945,Pathé,Astérix et Obélix: Mission Cléopatre
6,"Ewan McGregor, Natalie Portman, Samuel L. Jackson",113000000.0,John Williams,2005-05-18,146.0,3303005.0,Franchise,Fantasy,Etats-Unis,George Lucas,George Lucas,NaN,938,20th Century Fox,La Revanche des Sith
7,"Chris Evans, Chris Hemsworth, Robert Downey Jr...",356000000.0,Alan Silvestri,2019-04-24,181.0,3426471.0,Franchise,Comicbook,Etats-Unis,Kevin Feige,Russo (brothers),NaN,633,Walt Disney Pictures,Avengers: Endgame
10,"Sam Worthington, Sigourney Weaver, Kate Winslet",350000000.0,NaN,2022-12-14,192.0,2739848.0,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,762,Walt Disney Pictures,Avatar : la voie de l'eau
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9957,"Robert Pattinson, Kristen Stewart, Dakota Fanning",50000000.0,Alexandre Desplat,2009-11-18,130.0,2318559.0,Franchise,Fantasy,Etats-Unis,NaN,Chris Weitz,NaN,755,SND,Twilight - Chapitre 2 : Tentation
9959,"Viggo Mortensen, Ian McKellen, Elijah Wood, Ca...",92000000.0,Howard Shore,2002-12-18,179.0,2404825.0,Franchise,Fantasy,Etats-Unis,NaN,Peter Jackson,NaN,955,Metropolitan,Le Seigneur des anneaux: Les Deux tours
9960,"Vincent Cassel, Gérard Lanvin, Simon Pegg, - Q...",90000000.0,NaN,2009-07-03,87.0,2403734.0,Franchise,Animation,Etats-Unis,NaN,NaN,NaN,783,20th Century Fox,L'Age de glace 3 - Le Temps des dinosaures
9963,"Adam Driver, Harrison Ford, Samuel L. Jackson,...",275000000.0,John Williams,2019-12-18,142.0,2420710.0,Franchise,Fantasy,Etats-Unis,Kathleen Kennedy,J.J. Abrams,NaN,731,Walt Disney Pictures,Star Wars: L'Ascension de Skywalker


In [12]:
### SAUVEGARDER DATAFRAME EN CSV
movies_data_cleaned.to_csv('movies_clean_v1.csv', index=False)